<a href="https://colab.research.google.com/github/thayeylolu/cyberbullying/blob/lstm/notebooks/simple_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch import nn, save, load
from torch.optim import Adam
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch.nn.functional as F

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action="ignore")
%matplotlib inline

In [ ]:
def encode_label(label):
    if label == 'not_cyberbullying':
        return 0
    elif label == 'gender':
        return 1
    elif label == 'religion':
        return 2
    elif label == 'other_cyberbullying':
        return 3
    elif label == 'age':
        return 4
    else:
      return 5

In [ ]:
df = pd.read_csv('train_data.csv')
df= df.dropna()
df['clean_txt_emoji_lemma'] = df['clean_txt_emoji_lemma'].str.replace('\d+', '')


In [ ]:
df['clean_txt_emoji_lemma'].sample(3)

14970    make lot racist gay offensive joke page dming ...
20439    right tell idiots get socialism freedom vote e...
11805    racist joke religious joke abortion joke rape ...
Name: clean_txt_emoji_lemma, dtype: object

In [ ]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [ ]:
df['label'] = df['cyberbullying_type'].apply(encode_label)
df = df[['label', 'clean_txt_emoji_lemma']]

In [ ]:
df['label'].unique()

array([0, 1, 2, 3, 4, 5])

In [ ]:
train, test_valid = train_test_split(df,train_size= 0.70, random_state= 3040)
test, valid = train_test_split(test_valid,train_size= .50, random_state= 3040)
test['label'].value_counts()


2    1261
5    1215
0    1208
4    1164
1    1158
3    1070
Name: label, dtype: int64

In [ ]:
valid['label'].value_counts()

5    1238
2    1201
3    1172
1    1166
4    1156
0    1144
Name: label, dtype: int64

In [ ]:
train['label'].value_counts()

4    5672
1    5587
2    5535
5    5501
0    5417
3    5310
Name: label, dtype: int64

### VOcabuLARY BUILDING/


In [ ]:
X_train, Y_train = train['clean_txt_emoji_lemma'].to_list(), train['label'].to_list()
X_test, Y_test = test['clean_txt_emoji_lemma'].to_list(), test['label'].to_list()
X_valid, Y_valid = valid['clean_txt_emoji_lemma'].to_list(), valid['label'].to_list()

train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
test_dat=list(zip(Y_test,X_test))

In [ ]:
tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=[""])
vocab.set_default_index(vocab[""])

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [ ]:
text_pipeline('this is not a cyber bullying text')

[143, 0, 170, 0, 864, 0, 1334]

In [ ]:
## batch processing funciton

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)



In [ ]:
### text classification model

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5 #0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1])) #
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 10  # learning rate from 10 to 5 
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
test_iter2 =test_dat 
valid_iter2= valid_dat

train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

In [ ]:
cyber_label = {0:"not_cyberbullying",
                   1: "gender",
                   2: "religion",
                   3:"other_cyberbullying",
                   4: "age",
                   5: "ethnicity"
                  }
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()
 
ex_text_str = "Old people shouldnt be on social media"
model = model.to("cpu")

print("This is a %s tweet" %cyber_label[predict(ex_text_str, text_pipeline)])

In [ ]:
cyber_label = [0,1,2,3,4,5]
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()
 
ex_text_str = "Old people shouldnt be on social media"
model = model.to("cpu")

print("This is a %s tweet" %cyber_label[predict(ex_text_str, text_pipeline)])

## Model Evaluation
 on test data.

In [ ]:
test['predicted'] = test['clean_txt_emoji_lemma'].apply(lambda x : predict(x, text_pipeline))

In [ ]:
test

In [ ]:
# show columsn where label != predicted
compared_errors = test.loc[~(test['label'] == test['predicted'])]
compared_errors

In [ ]:
compared_errors['predicted'].value_counts()

In [ ]:
compared_errors['label'].value_counts()
 # 740 times `not cyberbullying was classifed wrongly`
# 290 times `gender was classfied wrongly`
# ...
# 50 times 'ethnicity` was clssified wrongly